In [13]:
import jieba.analyse,re
import jieba.posseg as pseg
import pandas 
corpus=pandas.read_csv("D:/电力短缺新闻/15tempdt.csv")["内容"].tolist()
def chinese_word_cut(mytext):
     jieba.load_userdict('电力短缺词典.txt');#加入自定义语料库
     words = pseg.cut(mytext)
     line_List=[]
     stopwords = open("C:/Users/张珍珍/project/DTMmodel/prepare/stopwords.txt",'r',encoding='utf-8').read().split('\n')
     #flags=['n','v','a','x','adv','vd','d','b']
     #flag_List=wordFlag.split(",")
     for w in words:
        i=0;
         #加入自定义语料库
        if(w.flag not in ['nt','ns','r','t','m','b','s','f','nr','nz','c'] and len(w.word)>1 and len(re.findall(r'[\u4e00-\u9fff]+',w.word))>0 and (not w.word.isnumeric()) and w.word not in stopwords and len(re.findall('\s',w.word))==0):
             line_List.append(w.word)
        i=i+1
     line=""
     for word in line_List:
          line+=word+" "
     return line
corpus=pandas.read_csv("D:/电力短缺新闻/15tempdt.csv")["内容"].tolist()
texts=[]
stopwords = open("C:/Users/张珍珍/project/DTMmodel/prepare/stopwords.txt",'r',encoding='utf-8').read()
stopwords = stopwords.split('\n')

In [14]:
for corpu in corpus:
    try:
        texts.append(chinese_word_cut(corpu))
    except:
        print(1)
texts1=[j.split(' ') for j in texts]

1


In [15]:
from gensim.models import Word2Vec
import jieba
import gensim, logging
# 文档集合
# 分词处理
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
vector_sizes=[64,128,256,320,384]
windows=[10,15,18]
hss=[0,1]
ns_exponents=[0.5,0.75,0.9]
alphas=[0.001,0.005,0.01,0.015,0.02,0.025,0.03,0.035,0.04,0.045,0.05,0.055,0.06,0.065,0.07,0.075,0.08,0.085,0.09,0.095,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
# 训练Skip-gram模型
min_counts=[10,15,20,25]
max_vocab_size=70000
samples=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
sorted_vocab=True
batch_words=[128,256,312,512]
compute_loss=True
sg=1
model = Word2Vec(texts1,vector_size = 256, min_count=15, window=10,sg=1)

In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
vocabulary = list(model.wv.key_to_index.keys())  # 获取词汇表
vectors = [model.wv[word] for word in vocabulary]  # 获取词向量
similarity_matrix = cosine_similarity(vectors)  # 计算余弦相似度矩阵
df = pd.DataFrame(similarity_matrix, columns=vocabulary, index=vocabulary)

In [19]:
import re
df2=pandas.read_excel("D:/电力短缺新闻/种子词.xlsx")
listz=[]
for j in set(df2["词语"][0:75]):
    list1=[]
    for i in set(df2["词语"][0:75]):
        try:
           list1.append(df.loc[j,i])
        except:
            list1.append('except')
    listz.append(list1)
df=pandas.DataFrame(listz)
df.index=df.columns=set(df2["词语"][0:75])
df.to_csv('D:/电力短缺新闻/余弦矩阵-cbow.csv',encoding='utf-8-sig')

In [23]:
from sklearn.manifold import MDS
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
mds = MDS(n_components=2, dissimilarity='precomputed')
embedded_coordinates = mds.fit_transform(df)

C:\Anaconda3\lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [24]:
import csv
with open('D:/电力短缺新闻/output-cbow.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(embedded_coordinates)

In [16]:
import pymongo
import pandas as pd
df=pd.read_csv("D:/电力短缺新闻/cbow-1.csv",encoding='utf-8-sig')
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["power-shortage"]
collection = db["cbow-1"]
for index, row in df.iterrows():
    mydict = {}
    for k, v in row.items():
        mydict[k] = v
    collection.insert_one(mydict)

In [44]:
import numpy as np

def label_propagation(adj_matrix, labels, alpha=0.9, max_iter=100):
    """
    标签传播算法
    :param adj_matrix: 转移概率矩阵，shape为(n, n)
    :param labels: 初始标签，shape为(n, num_classes)
    :param alpha: 随机游走的概率，默认为0.9
    :param max_iter: 最大迭代次数，默认为100
    :return: 更新后的标签，shape为(n, num_classes)
    """
    n = adj_matrix.shape[0]
    labels_new = np.copy(labels)
    for _ in range(max_iter):
        labels_propagated = alpha * adj_matrix.dot(labels_new)  # 标签传播
        labels_new = (1 - alpha) * labels_new + labels_propagated  # 更新标签
    return labels_new


# 测试代码
adj_matrix = np.array(df)
labels = np.array(listz)
updated_labels = label_propagation(adj_matrix, labels, alpha=0.8 ,max_iter=100)
print(updated_labels)

[[0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]
 [0.23115254 0.        ]


In [40]:
listz=[]
df2=pandas.read_excel("D:/电力短缺新闻/种子词.xlsx")
for j in df.columns:
    if(j in df2["词语"][0:20].tolist()):
        listz.append(1)
    else:
        listz.append(0)
listz

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0]

In [41]:
import numpy as np
def label_propagation(trans_matrix, labels, alpha=0.8, max_iter=500, tol=1e-4):
    num_nodes = len(trans_matrix)
    num_labels = len(labels)

    old_labels = np.copy(labels)
    iter_count = 0
    
    while iter_count < max_iter:
        iter_count += 1
        for i in range(num_nodes):
            labels = labels*trans_matrix
    return labels
# 示例转移概率矩阵和初始标签
df=pandas.read_csv("D:/电力短缺新闻/标签传播.csv",index_col='Unnamed: 0')
trans_matrix = np.array(df)
labels = np.array(listz)  # 初始标签
new_labels = label_propagation(trans_matrix, labels)
print("传播后的标签：", new_labels)

传播后的标签： [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
listz=[]
import pandas
df2=pandas.read_excel("D:/电力短缺新闻/种子词.xlsx")
for j in set(df2["词语"][0:75]):
    list1=[]
    for i in set(df2["词语"][0:75]):
        try:
           list1.append(df.loc[j,i])
        except:
            list1.append('except')
    listz.append(list1)